In [1]:
import warnings
warnings.filterwarnings('ignore')
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
sys.path.append('./../src/')
import dill
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
import matplotlib.pyplot as plt
from mplearn.feature_selection._adaptive_stable_minipatch_selection import AdaSTAMPS
from mplearn.feature_selection.base_selector import DecisionTreeSelector

import pandas as pd
import networkx as nx
import utils
import pickle
import numpy as np
import random
import io

random.seed(9)
np.random.seed(9)

date = "sep2024"
atleast = 2
subtree = nx.read_multiline_adjlist(path=f"uberon_{date}_atleast{atleast}_adjlist",create_using=nx.MultiDiGraph)
mlb = MultiLabelBinarizer().fit([[utils.id_to_name[node] for node in subtree.nodes]])

Mv_location = f"./../data/GEO/preprocessed/450K_Mvalues_atleast{atleast}_samplewise"
print(f"loading Mv, meta, mapping from {Mv_location}")
Mv, meta, mapping = dill.load(open(Mv_location, 'rb'))

le = LabelEncoder().fit(meta['tissue_name'].unique())
meta_le = le.transform(meta['tissue_name'].values)

print(meta_le.shape)
# print(le.classes_)

loading Mv, meta, mapping from ./../data/GEO/preprocessed/450K_Mvalues_atleast2_samplewise
(2604,)
